```Pridiction condition:
The target column (high_income) indicates a salary <= 50k per year (0), or > 50k per year (1)
algorythm: ID3 (Iterative Dichotomiser 3) ```

In [2]:
import pandas as pd
# Set index_col to False to avoid pandas thinking that the first column is row indexes (it's age)
income = pd.read_csv("../Raw_data/income.csv", index_col=False)
print(income.head(2))

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   

        marital_status        occupation    relationship    race    sex  \
0        Never-married      Adm-clerical   Not-in-family   White   Male   
1   Married-civ-spouse   Exec-managerial         Husband   White   Male   

   capital_gain  capital_loss  hours_per_week  native_country high_income  
0          2174             0              40   United-States       <=50K  
1             0             0              13   United-States       <=50K  


In [3]:
income["workclass"].value_counts()

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64

In [4]:
# convert categorical variables to numeric variables
col = pd.Categorical(income["workclass"])
income["workclass"] = col.codes
print(income["workclass"].value_counts())
# Convert columns from text categories to numbers
for name in ["education", "marital_status", "occupation", "relationship", "race", "sex", "native_country", "high_income"]:
    col = pd.Categorical(income[name])
    income[name] = col.codes

4    22696
6     2541
2     2093
0     1836
7     1298
5     1116
1      960
8       14
3        7
Name: workclass, dtype: int64


In [5]:
# Split income into two parts based on the value of the workclass column
# each data setscontaining the rows from one branch of the split
private_incomes = income[income['workclass']==4]
public_incomes = income[income['workclass'] !=4]
print(private_incomes.shape, public_incomes.shape)

(22696, 15) (9865, 15)


In [6]:
income["high_income"].value_counts()

0    24720
1     7841
Name: high_income, dtype: int64

In [7]:
income[income["high_income"] == 0].head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


In [8]:
# When splitting, the goal is to separate as many 0s from 1s in the high_income column.
# Using entrpy as a metric to measure how "together" the different values in the high_income column are

import math
# prob = lambda x: income[income["high_income"] == x].shape[0]/income.shape[0]
# a, b = prob(0), prob(1)
# entropy = -(a * math.log(a, 2) + b * math.log(b,2))
# compute the entropy of the high_income column

prob_0 = income[income["high_income"] == 0].shape[0] / income.shape[0]
prob_1 = income[income["high_income"] == 1].shape[0] / income.shape[0]
income_entropy = -(prob_0 * math.log(prob_0, 2) + prob_1 * math.log(prob_1, 2))
print('income_entropy: ', income_entropy)

income_entropy:  0.7963839552022132


In [9]:
# examine the age column
import numpy as np
np.bincount(income["age"])

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0, 395, 550, 712, 753, 720, 765, 877, 798, 841,
       785, 835, 867, 813, 861, 888, 828, 875, 886, 876, 898, 858, 827,
       816, 794, 808, 780, 770, 724, 734, 737, 708, 543, 577, 602, 595,
       478, 464, 415, 419, 366, 358, 366, 355, 312, 300, 258, 230, 208,
       178, 150, 151, 120, 108,  89,  72,  67,  64,  51,  45,  46,  29,
        23,  22,  22,  20,  12,   6,  10,   3,   1,   1,   3,   0,  43],
      dtype=int64)

In [10]:
income["age"].value_counts().describe()

count     73.000000
mean     446.041096
std      335.699081
min        1.000000
25%       72.000000
50%      419.000000
75%      785.000000
max      898.000000
Name: age, dtype: float64

In [11]:
'''
Compute the information gain for splitting on the age column of income.
    First, compute the median of age.
    Then, assign anything less than or equal to the median to the left branch, 
    and anything greater than the median to the right branch.
    Compute the information gain and assign it to age_information_gain.
'''
def calc_entropy(column):
    """ Calculate entropy given a pandas series, list, or numpy array. """
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
        
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    return -entropy

income_entropy = calc_entropy(income["high_income"])
median_age = income["age"].median()

left_split = income[income["age"] <= median_age]
right_split = income[income["age"] > median_age]

left_entropy = (left_split.shape[0] / income.shape[0]) * calc_entropy(left_split["high_income"])
right_entropy = (right_split.shape[0] / income.shape[0]) * calc_entropy(right_split["high_income"])

age_information_gain = income_entropy - (left_entropy + right_entropy)
print(f'age_information_gain: {age_information_gain}')

age_information_gain: 0.047028661304691965


In [12]:
# determine the best variable to split a node on

def calc_information_gain(data, split_name, target_name):
    """  Calculate information gain given a data set, column to split on, and target """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
       
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    return original_entropy - to_subtract

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]
information_gains =[]
for col in columns:
    infogain = calc_information_gain(income, col, "high_income")
    information_gains.append(infogain)
    
highest_gain_index = information_gains.index(max(information_gains))
highest_gain = columns[highest_gain_index]
print(f'highest_gain: {highest_gain}')

highest_gain: marital_status


#### Building a Decision Tree

In [13]:
# def id3(data, target, columns)
#     1 Create a node for the tree
#     2 If all values of the target attribute are 1, Return the node, with label = 1
#     3 If all values of the target attribute are 0, Return the node, with label = 0
#     4 Using information gain, find A, the column that splits the data best
#     5 Find the median value in column A
#     6 Split column A into values below or equal to the median (0), and values above the median (1)
#     7 For each possible value (0 or 1), vi, of A,
#     8    Add a new tree branch below Root that corresponds to rows of data where A = vi
#     9    Let Examples(vi) be the subset of examples that have the value vi for A
#    10    Below this new branch add the subtree id3(data[A==vi], target, columns)
#    11 Return Root

In [14]:
def find_best_column(data, target_name, columns):
    information_gains = []
    # Loop through and compute information gains
    for col in columns:
        information_gain = calc_information_gain(data, col, "high_income")
        information_gains.append(information_gain)

    # Find the name of the column with the highest gain
    highest_gain_index = information_gains.index(max(information_gains))
    highest_gain = columns[highest_gain_index]
    return highest_gain

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]
income_split = find_best_column(income, "high_income", columns)
print(f'income_split: {income_split}')

income_split: marital_status


In [15]:
# We'll use lists to store our labels for nodes (when we find them)
# Lists can be accessed inside our recursive function, whereas integers can't.  
# Look at the python missions on scoping for more information on this topic

import pandas as pd
label_1s = []
label_0s = []

def id3(data, target, columns):
    unique_targets = pd.unique(data[target])
    if len(unique_targets) == 1:
        if 0 in unique_targets:
            label_0s.append(0)
        elif 1 in unique_targets:
            label_1s.append(1)
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    
    for split in [left_split, right_split]:
        id3(split, target, columns)
    
data = pd.DataFrame([
    [0,20,0],
    [0,60,2],
    [0,40,1],
    [1,25,1],
    [1,35,2],
    [1,55,1]
    ])
data.columns = ["high_income", "age", "marital_status"]

id3(data, "high_income", ["age", "marital_status"])
print(label_0s, label_1s)

[0, 0, 0] [1, 1, 1]


In [16]:
# Create a dictionary to hold the tree
tree = {}
nodes = []
def id3(data, target, columns, tree):
    unique_targets = pd.unique(data[target])
    nodes.append(len(nodes) + 1)
    tree['number'] = nodes[-1]

    if len(unique_targets) == 1:
        if 0 in unique_targets:
            tree['label'] = 0
        elif 1 in unique_targets:
            tree['label'] = 1
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    tree['column'] = best_column
    tree['median'] = column_median
    
    left_split = data[data[best_column] <= column_median] 
    right_split = data[data[best_column] > column_median]
    split_dict = [['left', left_split], ['right', right_split]]
        
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])

id3(data, 'high_income', ['age', 'marital_status'], tree)
print(tree)

{'number': 1, 'column': 'age', 'median': 37.5, 'left': {'number': 2, 'column': 'age', 'median': 25.0, 'left': {'number': 3, 'column': 'age', 'median': 22.5, 'left': {'number': 4, 'label': 0}, 'right': {'number': 5, 'label': 1}}, 'right': {'number': 6, 'label': 1}}, 'right': {'number': 7, 'column': 'age', 'median': 55.0, 'left': {'number': 8, 'column': 'age', 'median': 47.5, 'left': {'number': 9, 'label': 0}, 'right': {'number': 10, 'label': 1}}, 'right': {'number': 11, 'label': 0}}}


In [17]:
def print_with_depth(string, depth):
    prefix = "    " * depth
    print("{0}{1}".format(prefix, string))

def print_node(tree, depth):
    if "label" in tree:
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        return
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    for branch in [tree["left"], tree["right"]]:
        print_node(branch, depth+1)

print_node(tree, 0)

age > 37.5
    age > 25.0
        age > 22.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 1
    age > 55.0
        age > 47.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 0


In [18]:
def predict(tree, row):
    if "label" in tree:
        return tree["label"]
    column = tree["column"]
    median = tree["median"]
    
    if row[column] <= median:
        predict(tree["left"], row)
        return tree["left"]
    return tree["right"]
print(predict(tree, data.iloc[0]))

{'number': 2, 'column': 'age', 'median': 25.0, 'left': {'number': 3, 'column': 'age', 'median': 22.5, 'left': {'number': 4, 'label': 0}, 'right': {'number': 5, 'label': 1}}, 'right': {'number': 6, 'label': 1}}


In [19]:
# Create a function named batch_predict() that takes two parameters, tree and df. 
new_data = pd.DataFrame([
    [40,0],
    [20,2],
    [80,1],
    [15,1],
    [27,2],
    [38,1]
    ])

new_data.columns = ["age", "marital_status"]

def batch_predict(tree, df):
    return df.apply(lambda x: predict(tree, x), axis=1)

predictions = batch_predict(tree, new_data)
print(predictions)

0    {'number': 7, 'column': 'age', 'median': 55.0,...
1    {'number': 2, 'column': 'age', 'median': 25.0,...
2    {'number': 7, 'column': 'age', 'median': 55.0,...
3    {'number': 2, 'column': 'age', 'median': 25.0,...
4    {'number': 2, 'column': 'age', 'median': 25.0,...
5    {'number': 7, 'column': 'age', 'median': 55.0,...
dtype: object


#### use scikit-learn package to fit a decision tree

In [20]:
from sklearn.tree import DecisionTreeClassifier

# A list of columns to train with
# We've already converted all columns to numeric
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

# Instantiate the classifier
# Set random_state to 1 to make sure the results are consistent
clf = DecisionTreeClassifier(random_state=1)
clf.fit(income[columns], income["high_income"])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [21]:
import numpy as np
import math

# Set a random seed so the shuffle is the same every time
np.random.seed(1)

income = income.reindex(np.random.permutation(income.index))

train_max_row = math.floor(income.shape[0] * .8)
train = income.iloc[:train_max_row]
test = income.iloc[train_max_row:]

In [22]:
from sklearn.metrics import roc_auc_score

clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
error = roc_auc_score(test["high_income"], predictions)
print(error)

0.6934656324746192


In [23]:
predictions = clf.predict(train[columns])
error = roc_auc_score(train["high_income"], predictions)
print(error)

0.9471244501437455


Overcome overfittting:
Restricting tree depth by adding a few parameters when initializing the DecisionTreeClassifier class:

    max_depth - Globally restricts how deep the tree can go
    min_samples_split - The minimum number of rows a node should have before it can be split; 
    min_samples_leaf - The minimum number of rows a leaf must have
    min_weight_fraction_leaf - The fraction of input rows a leaf must have
    max_leaf_nodes - The maximum number of total leaves, will cap the count of leaf nodes as the tree is being built

Some of these parameters aren't compatible, however. For example, we can't use max_depth and max_leaf_nodes together.

In [24]:
# Decision trees model from the last screen
clf = DecisionTreeClassifier(random_state=1, min_samples_split=13)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], predictions)
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)
print(test_auc)
print(train_auc)

0.6995617145150872
0.8421431849275413


In [25]:
# The first decision trees model we trained and tested
clf = DecisionTreeClassifier(random_state=1, max_depth=7, min_samples_split=13)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.7436344996725136
0.748037708309209


In [26]:
# A case of underfitting
clf = DecisionTreeClassifier(random_state=1, max_depth=2, min_samples_split=100)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.6553138481876499
0.6624508042161483


In [27]:
np.random.seed(1)
income["noise"] = np.random.randint(4, size=income.shape[0])

# Adjust "columns" to include the noise column
columns = ["noise", "age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

# Make new train and test sets
train_max_row = math.floor(income.shape[0] * .8)
train = income.iloc[:train_max_row]
test = income.iloc[train_max_row:]

# Initialize the classifier
clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], predictions)
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)
print(train_auc)
print(test_auc)

0.9750761614350801
0.6914060013941348
